In [1]:
# https://medium.com/analytics-vidhya/tutorial-on-text-classification-nlp-using-ulmfit-and-fastai-library-in-python-2f15a2aac065

In [2]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html 

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
  Could not find a version that satisfies the requirement torch_nightly (from versions: )
No matching distribution found for torch_nightly
You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install fastai

    100% |████████████████████████████████| 163kB 688kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 11.0MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/19/c6/39b29bc6c6c94298e81624746cc69184c9cf064de24d8c40aa68f9eb8779/spacy-2.0.18-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
    100% |████████████████████████████████| 112kB 701kB/s ta 0:00:01
    100% |████████████████████████████████| 61.4MB 680kB/s eta 0:00:01    71% |██████████████████████▉         | 43.9MB 3.5MB/s eta 0:00:06
  Using cached https://files.pythonhosted.org/packages/27/00/b2abd484bc4ef86e5c308acc6610e8f2c00c89b94701f42b9cc522ca70a4/thinc-6.12.1-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Running setup.py bdist_wheel for nvidia-ml-py3 ... done
  Stored in directory: /Users/grace/Library/Caches/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19

  Found existing installation: thinc 6.12.0
    Uninstalling thinc-6.12.0:
      Successfully uninstalled thinc-6.12.0
  Found existing installation: spacy 2.0.16
    Uninstalling spacy-2.0.16:
      Successfully uninstalled spacy-2.0.16
You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install dataclasses

You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import fastai 
from fastai import * 
from fastai.text import * 
import pandas as pd 
import numpy as np 
from functools import partial 
import io 
import os

In [6]:
from sklearn.datasets import fetch_20newsgroups 
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove= 
                             ('headers', 'footers', 'quotes'))
documents = dataset.data

In [7]:
documents[:10]

["Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 "\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap

In [8]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})
df.shape

(11314, 2)

In [9]:
df.head()

,label,text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."


In [0]:
# #df = df[df['label'].isin([1,10])] 
# df = df[df['label']<10]
# df = df.reset_index(drop = True)

In [10]:
df['label'].value_counts()

10    600
15    599
8     598
9     597
11    595
13    594
7     594
14    593
5     593
12    591
2     591
3     590
6     585
1     584
4     578
17    564
16    546
0     480
18    465
19    377
Name: label, dtype: int64

In [11]:
# data preprocessing

In [12]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [13]:
df.head()

,label,text
0,17,Well i m not sure about the story nad it did s...
1,0,Yeah do you expect people to read the ...
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,Well I will have to change the scoring on my ...


In [14]:
!pip install nltk

You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split()) 
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x:[item for item in x if 
                                    item not in stop_words]) 

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)):
    t =' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
df['text'] = detokenized_doc

[nltk_data] Downloading package stopwords to /Users/grace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from sklearn.model_selection import train_test_split 
# split data into training and validation set 
df_trn, df_val = train_test_split(df, stratify = df['label'], 
                                  test_size = 0.4, 
                                  random_state = 12)
df_trn.shape, df_val.shape

((6788, 2), (4526, 2))

In [17]:
# Language model data 
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = 
                                  df_val, path = "") 
# Classifier model data 
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, 
                                      valid_df = df_val,  
                                      vocab=data_lm.train_ds.vocab, 
                                      bs=32)

In [18]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103,  
                               drop_mult=0.7)

In [19]:
# train the learner object with learning rate = 1e-2 
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,6.262411,5.749133,0.221036


In [20]:
learn.save_encoder('ft_enc')

In [21]:
learn = text_classifier_learner(data_clas, drop_mult=0.7) 
learn.load_encoder('ft_enc')

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,2.590875,2.203593,0.304905


In [23]:
# get predictions 
preds, targets = learn.get_preds() 
predictions = np.argmax(preds, axis = 1) 
pd.crosstab(predictions, targets)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
row_0,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,37,55,34,38,23,17,1,2,1,0,6,27,1,3,0,2,0,0,0
3,0,28,43,120,86,11,58,7,1,0,0,9,46,0,2,0,1,0,0,0
4,0,0,0,0,3,0,3,0,1,0,0,0,2,0,1,0,0,0,0,0
5,1,79,48,21,19,118,6,3,3,1,0,23,24,9,5,3,0,2,0,0
6,1,16,21,10,13,6,76,12,13,3,3,7,10,2,4,0,4,1,1,0
7,0,2,0,2,8,0,7,75,70,0,0,0,10,0,8,0,1,0,1,0
8,0,0,1,0,0,0,0,7,15,0,0,0,0,0,1,0,1,0,0,0
9,0,0,0,1,0,1,0,4,2,11,6,0,0,0,3,0,1,0,0,0
